003に対して以下を追加.

- 損失関数への正則化項の追加: 論文の式(12)にある、グローバルアイテム埋め込みとユーザー固有のネガティブサンプリングアイテム埋め込み間の正則化項 
mathcalR(e_global,e_i −) を損失関数に含めます 。これにより、ローカルモデルがグローバルな知識から逸脱しすぎないように制約します
- ユーザー特徴抽出 MLP の導入: ユーザー特徴のより高次な表現を抽出するために、User Feature Refinement MLP を導入します 

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [16]:
# データセットの準備とクライアントへの分割 (1クライアント1ユーザー)
num_users = 100
num_items = 50
num_clients = num_users # 1クライアント1ユーザー

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
# ユーザーのインタラクション履歴はTransformer Blockがないため不要ですが、
# ダミーデータの整合性のために残します。
user_interaction_history = defaultdict(list) 

for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
            user_interaction_history[u_id].append(i_id) 
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = {} # {client_id: user_id}
client_datasets = {}
for u_id in range(num_users):
    client_id = u_id
    client_user_map[client_id] = u_id

    client_interactions_indices = [i for i, (u, _, _) in enumerate(interactions_list) if u == u_id]

    # Transformer Blockがないため、 historical_item_sequences は DataLoader に含めない
    user_interaction_data_for_client = []
    for idx in client_interactions_indices:
        u_id_data, i_id_data, label_data = interactions_list[idx]
        user_interaction_data_for_client.append((u_id_data, i_id_data, label_data))

    users_tensor = torch.tensor([d[0] for d in user_interaction_data_for_client], dtype=torch.long)
    items_tensor = torch.tensor([d[1] for d in user_interaction_data_for_client], dtype=torch.long)
    labels_tensor = torch.tensor([d[2] for d in user_interaction_data_for_client], dtype=torch.float32)

    client_subset = TensorDataset(users_tensor, items_tensor, labels_tensor)

    client_datasets[client_id] = DataLoader(client_subset, batch_size=32, shuffle=True)

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"Total interactions: {len(interactions)}")
print(f"Number of clients (1 client per user): {num_clients}")

Number of users: 100
Number of items: 50
Total interactions: 5000
Number of clients (1 client per user): 100


In [17]:
# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")

PLM embedding dimension: 384


In [18]:
class ClientModel(nn.Module):
    def __init__(self, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model

        # Joint Embedding Layer (module parameter θ_user) [cite: 64]
        # 論文の式(3) e_u = h(v_u) = v_u W_d1xd + b [cite: 78]
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)

        # Item Embedding Layer (module parameter θ_item) [cite: 64]
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # User Feature Refinement MLP (module parameter θ_umlp) [cite: 66]
        # 論文のImplementation Detailsで「user's mlp layer uses a two-layer mlp layer with a 32-64-32 architecture」とある [cite: 169]
        self.user_mlp = nn.Sequential(
            nn.Linear(joint_embedding_output_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32) # 出力次元はユーザー埋め込み次元と同じ32に設定 [cite: 169]
        )

        # Predictive Scoring Function (module parameter θ_score) [cite: 67]
        # 論文のImplementation Detailsで「32->16->8->1」のスキーマを持つMLPを使用とある [cite: 168]
        self.prediction_mlp = nn.Sequential(
            nn.Linear(item_embedding_dim + joint_embedding_output_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    # Transformer Block は削除されたため、historical_item_sequences は不要
    def forward(self, user_ids, item_ids, user_texts_batch): 
        # ユーザーのテキスト特徴をPLMで埋め込み
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :] # [CLS]トークンの埋め込みを使用

        # Joint Embedding Layer: テキスト埋め込みからユーザー特徴ベクトルを生成
        user_raw_embedding = self.user_joint_embedding_linear(plm_output)

        # User Feature Refinement MLP: ユーザー特徴の高次表現を抽出 [cite: 66]
        user_embedding = self.user_mlp(user_raw_embedding) # (batch_size, joint_embedding_output_dim)

        # アイテム埋め込み
        item_embedding = self.local_item_embedding(item_ids) # (batch_size, item_embedding_dim)

        # 予測層への入力は、User MLPの出力（ユーザー特徴）とターゲットアイテムの埋め込みを結合 [cite: 67]
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)
        logits = self.prediction_mlp(combined_features)
        predictions = torch.sigmoid(logits)

        # グラフ構築とアイテム集約のために、user_joint_embedding_linear.weight と local_item_embedding.weight を返す
        # 論文の「Parameter Uploading: Clients transmit user joint embedding weights and local item embeddings to the server.」 [cite: 61]
        return predictions, self.user_joint_embedding_linear.weight, self.local_item_embedding.weight

In [19]:
class Server:
    def __init__(self, num_users, num_items, item_embedding_dim, joint_embedding_output_dim):
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim

    def build_user_relationship_graph(self, user_linear_weights_map):
        """
        各ユーザーのuser_joint_embedding_linear.weightからユーザー関係グラフを構築します。
        論文の式 (15) に基づいています。 [cite: 106]

        Args:
            user_linear_weights_map (dict): {user_id: user_joint_embedding_linear.weight.data (d1, d2) Tensor}

        Returns:
            np.ndarray: ユーザーグラフの隣接行列 (NumPy配列)
            list: グラフのノード順に対応するユーザーIDのリスト
        """
        sorted_user_ids = sorted(user_linear_weights_map.keys())
        if not sorted_user_ids:
            return np.zeros((0, 0)), []

        # 各ユーザーの線形層の重みベクトルを収集
        # 論文の「w_i = vec(W_i)」に相当 [cite: 105]
        user_weight_vectors = np.array([
            user_linear_weights_map[u_id].flatten().cpu().numpy() for u_id in sorted_user_ids
        ])

        # コサイン類似度で類似度行列を計算 (S_ij) [cite: 106]
        similarity_matrix = cosine_similarity(user_weight_vectors)

        # ここでは簡単のため、完全な類似度グラフを使用 (S' に相当) [cite: 108]
        # 論文の「take the top-N in the highest similarity list」 は、後のステップで実装可能 [cite: 108]
        user_graph_adj = similarity_matrix

        return user_graph_adj, sorted_user_ids

    def aggregate_item_embeddings(self, user_local_item_weights, user_graph_adj, sorted_user_ids):
        """
        ユーザー関係グラフに基づいて、アイテム埋め込みをグローバルに集約します。
        論文の式 (16) と (17) に基づいています。 [cite: 111, 114]

        Args:
            user_local_item_weights (dict): {user_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するユーザーIDのリスト

        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """

        # グラフの順序に合わせて各ユーザーのアイテム埋め込みを行列Aとしてまとめる
        # A は (num_users, num_items, item_embedding_dim)
        # 論文の「A is the round item embedding matrix, the I-th row represents the item embedding obtained from user i」に相当 [cite: 111]
        item_embedding_matrix_A = torch.stack([
            user_local_item_weights[u_id] for u_id in sorted_user_ids
        ]) # (num_users, num_items, item_embedding_dim)

        # グラフの正規化 (S'')
        # NOTE: 正規化が必要な理由
        ### スケール調整と数値安定性: 隣接行列をそのまま使うと、ノードの次数（接続数）が大きいほど、そのノードから受け取る情報の合計が非常に大きくなってしまいます。これにより、特徴量のスケールが大きくなりすぎたり、訓練中に勾配爆発を引き起こしたりする可能性があります。正規化は、この影響を均一化し、数値的な安定性を確保するのに役立ちます 。
        ### 特徴量の平滑化と拡散: 正規化は、ノードの特徴量（この場合はアイテム埋め込み）が隣接ノードに適切に伝播・拡散されることを保証します。正規化されていない場合、高次数のノードが支配的になり、低次数のノードの情報が埋もれてしまう可能性があります。
        ### GCNの理論的根拠: LightGCNのようなGCNモデルでは、グラフ畳み込み操作がグラフ上の情報の平滑化と拡散として機能します 。正規化は、この平滑化プロセスが効果的に機能するために不可欠です。
        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1
        normalized_user_graph_adj = user_graph_adj / row_sums_graph

        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        # グラフ畳み込み (R = S'' A) [cite: 111]
        # R は (num_users, num_items, item_embedding_dim) となる
        # MatMul: (num_users, num_users) x (num_users, num_items, item_embedding_dim)
        # Einstein Summation Convention: 'ij, jkd -> ikd'
        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, item_embedding_matrix_A)

        # グローバルアイテム埋め込みの更新 (θ_global = DR) [cite: 113, 114]
        # ここではDを全ユーザーの単純平均と解釈 (Rの0次元目を平均)
        new_global_item_embedding_weight = R_tensor.mean(dim=0) # (num_items, item_embedding_dim)

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_weight)

        return self.global_item_embedding.weight.data

In [ ]:
# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    )
    # NOTE:
    # クライアントごとに最適化するパラメータを設定
    # ここでは、user_joint_embedding_linear, local_item_embedding, prediction_layer が対象
    # 単純にoptim.Adam(params = client_models[client_id].parameters(), lr=0.001)とすると、
    # PLMも学習可能パラメータとなってしまうので、
    # PLMのパラメータを除外したパラメータのみを取得してから、設定する.
    trainable_params = [
        p for name, p in client_models[client_id].named_parameters()
        if not name.startswith('plm_model.')
    ]

    client_optimizers[client_id] = optim.Adam(
        params=trainable_params,
        lr=0.001
    )

# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1

for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")

    # サーバーからグローバルアイテム埋め込みをクライアントに配布 [cite: 63]
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    user_linear_weights_for_graph = {} # {user_id: user_joint_embedding_linear.weight.data (d1, d2) Tensor}
    user_local_item_weights_to_server = {} # {user_id: local_item_embedding.weight.data (Tensor)}

    # クライアントのローカル学習 [cite: 60]
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]

        model.train()
        local_loss = 0

        current_user_id = client_user_map[client_id] 

        if len(dataloader.dataset) == 0:
            print(f"  Client {client_id} (User {current_user_id}) has no interactions, skipping local training.")
            # 訓練されなかったクライアントのために、現在のモデルの重み（グローバル初期化時と同じ）をアップロード
            user_linear_weights_for_graph[current_user_id] = model.user_joint_embedding_linear.weight.data.clone().flatten()
            user_local_item_weights_to_server[current_user_id] = model.local_item_embedding.weight.data.clone()
            continue

        for epoch in range(local_epochs):
            # Transformer Blockがないため、 historical_item_sequences は DataLoader から取得しない
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:
                assert torch.all(user_ids_batch == current_user_id) 
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]

                optimizer.zero_grad()
                # historical_item_sequences_batch を渡さない
                predictions, user_joint_embedding_linear_weight, local_item_embedding_weight = model(
                    user_ids_batch, item_ids_batch, current_user_texts
                )

                # 損失計算 (L_1のみ) [cite: 90, 91]
                loss = nn.BCELoss()(predictions.squeeze(), labels_batch)

                # 正則化項の追加 (論文の式(11)と(12)) [cite: 92, 93, 95]
                # ここではe_globalはサーバーのglobal_item_embedding.weight.data
                # e_i^- はlocal_item_embedding_weightからネガティブサンプリングされたアイテム埋め込み
                # 論文では「Mean((e_global - e_i^-)^2)」 [cite: 93]
                # 正確な e_i^- のサンプリングはデータセットからのネガティブサンプリングロジックが必要だが、ここでは簡略化
                lambda_reg = 0.01 # ハイパーパラメータ [cite: 96, 101]

                # global_item_embeddingとlocal_item_embeddingのL2距離を正則化項とする
                # (ネガティブサンプリングは省略)
                regularization_term = torch.mean(
                    (local_item_embedding_weight - server.global_item_embedding.weight.data)**2
                )

                loss = loss + lambda_reg * regularization_term

                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        # クライアントがサーバーにアップロードするパラメータを収集 [cite: 61]
        user_linear_weights_for_graph[current_user_id] = user_joint_embedding_linear_weight.data.clone()
        user_local_item_weights_to_server[current_user_id] = local_item_embedding_weight.data.clone()

        print(f"  Client {client_id} (User {current_user_id}) local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    # 論文のステップ「Graph Aggregation: The server constructs user relation graphs from text embeddings and aggregates parameters through graph convolution.」 [cite: 62]
    # ユーザー関係グラフの構築 [cite: 103]
    user_graph_adj, sorted_user_ids_for_graph = server.build_user_relationship_graph(
        user_linear_weights_for_graph
    )
    
    # アイテム埋め込みの集約 [cite: 109, 110]
    server.aggregate_item_embeddings(
        user_local_item_weights_to_server, 
        user_graph_adj, 
        sorted_user_ids_for_graph
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")


--- Communication Round 1/10 ---
  Client 0 (User 0) local loss: 0.7057
  Client 1 (User 1) local loss: 0.6584
  Client 2 (User 2) local loss: 0.6494
  Client 3 (User 3) local loss: 0.7032
  Client 4 (User 4) local loss: 0.7420
  Client 5 (User 5) local loss: 0.7785
  Client 6 (User 6) local loss: 0.7540
  Client 7 (User 7) local loss: 0.7494
  Client 8 (User 8) local loss: 0.7111
  Client 9 (User 9) local loss: 0.7557
  Client 10 (User 10) local loss: 0.7830
  Client 11 (User 11) local loss: 0.6777
  Client 12 (User 12) local loss: 0.6591
  Client 13 (User 13) local loss: 0.6994
  Client 14 (User 14) local loss: 0.7935
  Client 15 (User 15) local loss: 0.7966
  Client 16 (User 16) local loss: 0.6800
  Client 17 (User 17) local loss: 0.6641
  Client 18 (User 18) local loss: 0.6642
  Client 19 (User 19) local loss: 0.6925
  Client 20 (User 20) local loss: 0.6919
  Client 21 (User 21) local loss: 0.7463
  Client 22 (User 22) local loss: 0.6502
  Client 23 (User 23) local loss: 0.7519
  